In [121]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
movies = pd.read_csv("movies.csv")
ratings = pd.read_csv("ratings.csv")

In [3]:
movies.head()

,movieId,title,genres,imdbId,poster_img
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,https://m.media-amazon.com/images/M/MV5BMDU2ZW...
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,https://m.media-amazon.com/images/M/MV5BZTk2Zm...
2,3,Grumpier Old Men (1995),Comedy|Romance,113228,https://m.media-amazon.com/images/M/MV5BMjQxM2...
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,https://m.media-amazon.com/images/M/MV5BYzcyMD...
4,5,Father of the Bride Part II (1995),Comedy,113041,https://m.media-amazon.com/images/M/MV5BOTEyNz...


In [4]:
ratings.head()

,userId,movieId,rating
0,3,1,4.0
1,3,29,4.5
2,3,32,4.5
3,3,50,5.0
4,3,111,4.0


In [22]:
user_movie_id = [201588,115149,168248,1721,152081,122914,122912,2059]
user_rating = [5,2,1,4.5,4,5,4.5,4]
user_id = [1000000,1000000,1000000,1000000,1000000,1000000,1000000,1000000]

In [29]:
my_rating = pd.DataFrame({'userId' : user_id, 'movieId': user_movie_id,'rating' : user_rating})
my_rating

,userId,movieId,rating
0,1000000,201588,5.0
1,1000000,115149,2.0
2,1000000,168248,1.0
3,1000000,1721,4.5
4,1000000,152081,4.0
5,1000000,122914,5.0
6,1000000,122912,4.5
7,1000000,2059,4.0


In [33]:
new_rating = pd.concat([ratings,my_rating],axis=0,ignore_index=True)

In [35]:
rat_pivot = new_rating.pivot(index = "userId",columns = "movieId",values = "rating")
# rat_pivot = rat_pivot.fillna(0)
rat_pivot

movieId,1,2,3,4,5,6,7,8,9,10,...,202103,202393,202429,202439,203222,203519,204352,204542,204698,205383
userId,,,,,,,,,,,,,,,,,,,,,
3,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,4.0,2.0,2.0,NaN,NaN,NaN,3.0,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,NaN,2.0,NaN,NaN,NaN,4.0,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,NaN,1.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162495,3.0,3.0,3.5,NaN,2.0,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162508,4.5,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162516,4.5,2.5,0.5,2.0,NaN,4.5,2.0,NaN,NaN,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [97]:
def standardize(row):
    return (row - row.mean())/(row.max()-row.min())

In [159]:
user_movie_ratings = rat_pivot[user_movie_id]
user_movie_ratings = user_movie_ratings.fillna(0)
user_movie_ratings = user_movie_ratings_1.apply(standardize)
user_similarity = cosine_similarity(user_movie_ratings)
user_similarity = pd.DataFrame(user_similarity,columns = rat_pivot.index,index = rat_pivot.index)
my_row = user_similarity.iloc[-1,:]
my_row = my_row.sort_values(ascending = False)
most_similar = my_row.iloc[1:6]
most_similar_user = most_similar.index.to_list()
most_similar_score = list(my_row.iloc[1:6].values)
not_watched = rat_pivot.drop(columns = user_movie_id)
not_watched = not_watched.T[most_similar_user]
not_watched = not_watched[not_watched.isnull().sum(axis=1) < 3]
my_dict=dict()
for i in range(len(most_similar_user)):
    my_dict[most_similar_user[i]] = most_similar_score[i]
for i in most_similar_user:
    not_watched[i]*=my_dict[i]
score = []
for i in range(not_watched.shape[0]):
    s1 = 0
    s2 = 0
    for j in range(not_watched.shape[1]):
        if(np.isnan(not_watched.iloc[i,j])):
            continue
        else:
            s1 = s1+not_watched.iloc[i,j]
            s2 = s2 + my_dict[not_watched.columns[j]]
            
    s1 = s1/s2
    score.append(s1)
not_watched['score'] = score
not_watched = not_watched.sort_values(by = "score",ascending = False)

In [163]:
movie_list = not_watched.iloc[:10,-1]
movies.loc[movies['movieId'].isin(movie_list.index.to_list())]

,movieId,title,genres,imdbId,poster_img
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,https://m.media-amazon.com/images/M/MV5BZTk2Zm...
1525,1907,Mulan (1998),Adventure|Animation|Children|Comedy|Drama|Musi...,120762,https://m.media-amazon.com/images/M/MV5BODkxNG...
3500,4306,Shrek (2001),Adventure|Animation|Children|Comedy|Fantasy|Ro...,126029,https://m.media-amazon.com/images/M/MV5BOGZhM2...
4019,4993,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy,120737,https://m.media-amazon.com/images/M/MV5BN2EyZj...
5298,7153,"Lord of the Rings: The Return of the King, The...",Action|Adventure|Drama|Fantasy,167260,https://m.media-amazon.com/images/M/MV5BNzA5ZD...
7783,72998,Avatar (2009),Action|Adventure|Sci-Fi|IMAX,499549,https://m.media-amazon.com/images/M/MV5BMTYwOT...
9120,122918,Guardians of the Galaxy 2 (2017),Action|Adventure|Sci-Fi,3896198,https://m.media-amazon.com/images/M/MV5BNjM0NT...
9578,163645,Hacksaw Ridge (2016),Drama|War,2119532,https://m.media-amazon.com/images/M/MV5BMjQ1Nj...
9684,170645,The Zookeeper's Wife (2017),Drama,1730768,https://m.media-amazon.com/images/M/MV5BNTY3Ym...
9977,200540,Aladdin (2019),Adventure|Fantasy|Romance,6139732,https://m.media-amazon.com/images/M/MV5BMjQ2OD...
